<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/qrtopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds

     |████████████████████████████████| 3.4 MB 7.8 MB/s 


In [2]:
import wrds

In [3]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [4]:
db.describe_table('optionm','opvold')

Approximately 61360236 rows in optionm.opvold.


,name,nullable,type
0,secid,True,DOUBLE_PRECISION
1,date,True,DATE
2,cp_flag,True,VARCHAR(1)
3,volume,True,DOUBLE_PRECISION
4,open_interest,True,DOUBLE_PRECISION


In [5]:
tqry = """
        select * from optionm.opvold
        where date between '2017/01/01' and '2017/12/31' and
           cp_flag is null        
"""

cpqry = """
        select * from optionm.opvold
        where date between '2017/01/01' and '2017/12/31' and
           cp_flag is not null
"""

tvol = db.raw_sql(tqry,date_cols='date')
cpvol = db.raw_sql(cpqry,date_cols='date')

In [6]:
tvol.secid = tvol.secid.astype(int)
cpvol.secid = cpvol.secid.astype(int)

In [7]:
tvol.rename({'volume':'tvol','open_interest':'tinterest'},
             axis=1,inplace=True)

In [8]:
list(cpvol)

['secid', 'date', 'cp_flag', 'volume', 'open_interest']

In [9]:
cpvol = cpvol.pivot_table(index=['secid','date'],columns='cp_flag',
                  values=['volume','open_interest'])

In [10]:
cpvol.columns = ['_'.join(col) for col in cpvol.columns.values]

In [11]:
tvol.drop('cp_flag',inplace=True,axis=1)

In [12]:
cpvol['int_gap'] = cpvol['open_interest_C']- cpvol['open_interest_P']

In [13]:
cpvol.reset_index()

,secid,date,open_interest_C,open_interest_P,volume_C,volume_P,int_gap
0,6505,2017-01-03,28.0,229.0,0.0,0.0,-201.0
1,6505,2017-01-04,28.0,229.0,0.0,2.0,-201.0
2,6505,2017-01-05,28.0,229.0,0.0,0.0,-201.0
3,6505,2017-01-06,28.0,229.0,0.0,1.0,-201.0
4,6505,2017-01-09,28.0,230.0,0.0,12.0,-202.0
...,...,...,...,...,...,...,...
1085601,212409,2017-12-22,938.0,110.0,27.0,20.0,828.0
1085602,212409,2017-12-26,951.0,130.0,22.0,33.0,821.0
1085603,212409,2017-12-27,969.0,163.0,14.0,53.0,806.0
1085604,212409,2017-12-28,970.0,212.0,13.0,2094.0,758.0


In [14]:
tvol = tvol.set_index(['secid','date']).sort_index().reset_index()

In [17]:
tvol.to_stata('tvol.dta')
cpvol.to_stata('cpvol.dta')